In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import skimage.measure
import random
import copy
import math
import cv2
import os

# ------------------------
# Configuration Parameters
# ------------------------

div = 64  # Size of the square crop windows
red = 1   # Reduction factor
resize_ratio = 1  # Ratio to resize maps
n_crops = 300  # Number of random crops per map
chans = 13    # Number of semantic classes

# Box filter kernels
box = 4
kernel = np.ones((box,box),np.float32)/(box**2)
boxp = 8
kernelp = np.ones((boxp,boxp),np.float32)/(boxp**2)

# ------------------------
# Main Processing Function
# ------------------------

def process_map(map_name):
    """
    Extracts training samples from a single map.

    Args:
        map_name (str): Name of the map (e.g., 'stanford_bookstore0')
    """

    print(map_name)

    # Training data initialization
    train_x = np.zeros((1, div, div, chans))
    train_y = np.zeros((1, div, div))
    train_y1 = np.zeros((1, div, div))
    train_y2 = np.zeros((1, div, div))

    train_data_dir = '/data/placido/training_data'

    # ------------------------
    # Load Maps
    # ------------------------

    hd_path = f'/home/placido.falqueto/IRI_Barcelona/maps/13semantics/{map_name}/humandensity-{map_name}-new.csv'
    vel_path = f'/home/placido.falqueto/IRI_Barcelona/maps/13semantics/{map_name}/humandensity-{map_name}-vel.csv'
    stop_path = f'/home/placido.falqueto/IRI_Barcelona/maps/13semantics/{map_name}/humandensity-{map_name}-stop.csv'
    png_path = f'/home/placido.falqueto/IRI_Barcelona/maps/13semantics/{map_name}/{map_name}_colors.png'
    labelmap_path = '/home/placido.falqueto/IRI_Barcelona/maps/13semantics/labelmap.txt'

    # Load human density, velocity, and stop maps
    data = np.genfromtxt(hd_path, delimiter=',')
    data1 = np.genfromtxt(vel_path, delimiter=',')
    data2 = np.genfromtxt(stop_path, delimiter=',')

    # Downsample maps
    data = skimage.measure.block_reduce(data, (red, red), np.max)
    data1 = skimage.measure.block_reduce(data1, (red, red), np.max)
    data2 = skimage.measure.block_reduce(data2, (red, red), np.max)

    # Resize maps if needed
    data = cv2.resize(data, None, fx=resize_ratio, fy=resize_ratio, interpolation=cv2.INTER_LINEAR)
    data1 = cv2.resize(data1, None, fx=resize_ratio, fy=resize_ratio, interpolation=cv2.INTER_LINEAR)
    data2 = cv2.resize(data2, None, fx=resize_ratio, fy=resize_ratio, interpolation=cv2.INTER_LINEAR)

    # Blur data
    data = cv2.filter2D(data,-1,kernel)
    data = cv2.filter2D(data,-1,kernel)
    data1 = cv2.filter2D(data1,-1,kernel)
    data1 = cv2.filter2D(data1,-1,kernel)
    data2 = cv2.filter2D(data2,-1,kernelp)
    data2 = cv2.filter2D(data2,-1,kernelp)

    hd, wd = data.shape

    # ------------------------
    # Create Semantic Map
    # ------------------------

    image = cv2.imread(png_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (wd, hd), interpolation=cv2.INTER_LINEAR)

    map = np.zeros((hd, wd, chans))
    colors = []

    with open(labelmap_path, 'r') as f:
        labelmap = [line.strip().split(':') for line in f.readlines()[2:]]

    for i, label in enumerate(labelmap):
        label_color = np.array(label[1].split(','), dtype=int)
        colors.append(label_color)
        map[:, :, i] = 255 - cv2.inRange(image, label_color - 10, label_color + 10)

    # Load and process shade map if available
    if os.path.isfile(f'/home/placido.falqueto/IRI_Barcelona/maps/13semantics/{map_name}/{map_name}_shades.png'):
        png_path = '/home/placido.falqueto/IRI_Barcelona/maps/13semantics/'+map_name+'/'+map_name+'_shades.png'
        # Load the PNG image
        image = cv2.imread(png_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (wd,hd), interpolation = cv2.INTER_LINEAR)
        map[:,:,-1] = 255-cv2.inRange(image, label_color-10, label_color+10)

    map = map / 255 # Normalize map channels

    # print(map.shape)
    h, w, _ = map.shape

    sem_map = np.zeros((h,w,3))
    for i in range(chans):
        sem = np.full((h,w,3),colors[i])
        sem_map = np.stack((1-map[:,:,i],1-map[:,:,i],1-map[:,:,i]), axis=2)*sem+sem_map
    
    plt.figure(figsize=(5,5))
    plt.imshow(sem_map/255)
    plt.axis('off')
    plt.show()


    # ------------------------
    # Random Crop Generation
    # ------------------------

    selections = np.zeros((hd, wd))  
    crops = np.zeros((n_crops, 2), dtype=int)

    for i in range(n_crops):
        while True:
            aux_x = int(random.random() * (wd - div + 1))
            aux_y = int(random.random() * (hd - div + 1))

            submap = map[aux_y:aux_y + div, aux_x:aux_x + div, :]
            subdata = data[aux_y:aux_y + div, aux_x:aux_x + div]
            subdata1 = data1[aux_y:aux_y + div, aux_x:aux_x + div]
            subdata2 = data2[aux_y:aux_y + div, aux_x:aux_x + div]

            # Check if the crop contains enough information
            if np.mean(submap) > 0.2 and np.mean(submap) < 1 and np.mean(subdata) > 0 and submap.shape == (div, div, chans):
                crops[i, :] = (aux_x, aux_y)
                selections += cv2.rectangle(np.zeros_like(selections), (crops[i, 0], crops[i, 1]), (crops[i, 0] + div - 1, crops[i, 1] + div - 1), (1, 0, 0), -1)
                train_x = np.append(train_x, np.expand_dims(submap, axis=0), axis=0)
                train_y = np.append(train_y, np.expand_dims(subdata, axis=0), axis=0)
                train_y1 = np.append(train_y1, np.expand_dims(subdata1, axis=0), axis=0)
                train_y2 = np.append(train_y2, np.expand_dims(subdata2, axis=0), axis=0)
                break  

    # Calculate weights with protection against division by zero
    epsilon = 1e-6  # A small value to avoid division by zero
    selections = 1.0 / (selections + epsilon)

    data_pred = np.zeros((int(math.ceil(hd)),int(math.ceil(wd))))
    data_pred1 = np.zeros((int(math.ceil(hd)),int(math.ceil(wd))))
    data_pred2 = np.zeros((int(math.ceil(hd)),int(math.ceil(wd))))
    for i in range(n_crops):
        submap = map[crops[i,1]:crops[i,1]+div, crops[i,0]:crops[i,0]+div, :]
        subdata = data[crops[i,1]:crops[i,1]+div, crops[i,0]:crops[i,0]+div]
        subdata1 = data1[crops[i,1]:crops[i,1]+div, crops[i,0]:crops[i,0]+div]
        subdata2 = data2[crops[i,1]:crops[i,1]+div, crops[i,0]:crops[i,0]+div]
        data_pred[crops[i,1]:crops[i,1]+div, crops[i,0]:crops[i,0]+div] += subdata*selections[crops[i,1]:crops[i,1]+div, crops[i,0]:crops[i,0]+div]
        data_pred1[crops[i,1]:crops[i,1]+div, crops[i,0]:crops[i,0]+div] += subdata1*selections[crops[i,1]:crops[i,1]+div, crops[i,0]:crops[i,0]+div]
        data_pred2[crops[i,1]:crops[i,1]+div, crops[i,0]:crops[i,0]+div] += subdata2*selections[crops[i,1]:crops[i,1]+div, crops[i,0]:crops[i,0]+div]

    plt.figure(figsize=(5,5))
    plt.imshow(data_pred)
    plt.axis('off')
    plt.show()

    plt.figure(figsize=(5,5))
    plt.imshow(data_pred1)
    plt.axis('off')
    plt.show()

    plt.figure(figsize=(5,5))
    plt.imshow(data_pred2)
    plt.axis('off')
    plt.show()
    

    # ------------------------
    # Data Augmentation
    # ------------------------

    for i in range(len(crops)): 
        for j in range(3):  # Rotations
            train_x = np.append(train_x, np.expand_dims(np.rot90(train_x[i,:,:], k=j+1, axes=(0, 1)),axis=0), axis=0)
            train_y = np.append(train_y, np.expand_dims(np.rot90(train_y[i,:,:], k=j+1, axes=(0, 1)),axis=0), axis=0)
            train_y1 = np.append(train_y1, np.expand_dims(np.rot90(train_y1[i,:,:], k=j+1, axes=(0, 1)),axis=0), axis=0)
            train_y2 = np.append(train_y2, np.expand_dims(np.rot90(train_y2[i,:,:], k=j+1, axes=(0, 1)),axis=0), axis=0)

        # Flipping
        train_x = np.append(train_x, [np.flip(submap, axis=0)], axis=0)
        train_y = np.append(train_y, [np.flip(subdata, axis=0)], axis=0)
        train_y1 = np.append(train_y1, [np.flip(subdata1, axis=0)], axis=0)
        train_y2 = np.append(train_y2, [np.flip(subdata2, axis=0)], axis=0)

        train_x = np.append(train_x, [np.flip(submap, axis=1)], axis=0)
        train_y = np.append(train_y, [np.flip(subdata, axis=1)], axis=0)
        train_y1 = np.append(train_y1, [np.flip(subdata1, axis=1)], axis=0)
        train_y2 = np.append(train_y2, [np.flip(subdata2, axis=1)], axis=0)

    # ------------------------
    # Prepare Training Data
    # ------------------------

    train_x = np.delete(train_x, 0, 0)  # Remove placeholder row
    train_y = np.delete(train_y, 0, 0)
    train_y1 = np.delete(train_y1, 0, 0)
    train_y2 = np.delete(train_y2, 0, 0)

    print(f'Final shapes: train_x: {train_x.shape}, train_y: {train_y.shape}')

    # ------------------------
    # Save Training Data
    # ------------------------

    output_dir = f'{train_data_dir}/{div}crop_size/{chans}labels/{red}red/{map_name}'
    os.makedirs(output_dir, exist_ok=True)

    np.savetxt(os.path.join(output_dir, 'train_X.csv'), np.insert(train_x.flatten(), 0, train_x.shape), delimiter=',', fmt='%f')
    np.savetxt(os.path.join(output_dir, 'train_Y.csv'), np.insert(train_y.flatten(), 0, train_y.shape), delimiter=',', fmt='%f')
    np.savetxt(os.path.join(output_dir, 'train_Y1.csv'), np.insert(train_y1.flatten(), 0, train_y1.shape), delimiter=',', fmt='%f')
    np.savetxt(os.path.join(output_dir, 'train_Y2.csv'), np.insert(train_y2.flatten(), 0, train_y2.shape), delimiter=',', fmt='%f')

# ------------------------
# Main Loop
# ------------------------

if __name__ == "__main__":
    map_list = [
        'stanford_bookstore0', 'stanford_bookstore4', 'stanford_coupa0', 'stanford_coupa1',
        'stanford_coupa2', 'stanford_coupa3', 'stanford_gates0', 'stanford_gates1',
        'stanford_gates2', 'stanford_gates3', 'stanford_hyang2', 'stanford_hyang3',
        'stanford_hyang4',  'stanford_hyang10', 'stanford_little3', 'stanford_nexus0',
        'stanford_nexus1',  'stanford_deathCircle0'
    ]

    for map_name in map_list:
        process_map(map_name)


